# Embedding of the Bible Interpreted

### Flattening the json

In [ ]:
# Importing library
import json
import pandas as pd
import os

# Setting directory
# os.chdir('root_repository_path')

# Importing data
#with open('./bible/data/bible_by_theme_int.json') as f:
#    data = json.load(f)

In [37]:
# Creating empty lists to save the data
list_id = []
list_purpose = []
list_passage = []
list_text = []
list_interpretation = []
list_area_life = []

# For loop to interpret all the fragments of the Bible
i = 0
for item in data:
    purpose = item['purpose']
    verses = item['versículos']
    for verse in verses:
        # Generating and id for each Bible verse
        i=i+1
        id = "vec"+str(i)
        list_id.append(id)
        passage = verse['pasaje']
        text = verse['texto']
        interpretation = verse['interpretación']
        area_life = verse['áreas de la vida']
        list_purpose.append(purpose)
        list_passage.append(passage)
        list_text.append(text)
        list_interpretation.append(interpretation)
        list_area_life.append(area_life)

In [38]:
# Creating a list to store the data
flat_data = []

# Iterate through the lists and populate the list
for id, purpose, passage, text, interpretation, area in zip(list_id, list_purpose, list_passage, list_text, list_interpretation, list_area_life):
    flat_data.append({
        'id': id,
        'tema': purpose,
        'versiculo': passage,
        'texto': text,
        'interpretacion': interpretation,
        'area_vida': area
    })


# Save the list as a JSON file
with open('bible/data/bible_by_theme_int_flat.json', 'w', encoding='utf-8') as json_file:
   json.dump(flat_data, json_file, ensure_ascii=False, indent=4)

### Creating embeddings

In [41]:
# Importing library
import json
import pandas as pd
import os

# Setting directory
# os.chdir('root_repository_path')

with open('bible/data/bible_by_theme_int_flat.json') as f:
    flat_data = json.load(f)

# Converting json into df
bible_data = pd.DataFrame(flat_data)

In [17]:
# Setting the libraries and connection
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# Loading api_key
openai.api_key = os.getenv('OPENAI_API_KEY')

In [42]:
bible_data.head()

,id,tema,versiculo,texto,interpretacion,area_vida
0,vec1,salvación,Mateo 18:3,"Y dijo: De cierto os digo, que si no os volvie...",Jesús está diciendo que debemos tener la humi...,"[Desarrollo/crecimiento personal, Misión de vida]"
1,vec2,salvación,Juan 1:12,"Mas á todos los que le recibieron, dióles pote...",Este versículo significa que a todas las pers...,"[Desarrollo/crecimiento personal, Misión de vida]"
2,vec3,salvación,Juan 3:3,"Respondió Jesús, y díjole: De cierto, de ciert...",Jesús está hablando de la necesidad de un ren...,"[Desarrollo/crecimiento personal, Misión de vida]"
3,vec4,salvación,Juan 3:16,"Porque de tal manera amó Dios al mundo, que ha...",Este versículo expresa el amor inmenso de Dio...,"[Misión de vida, Desarrollo/crecimiento person..."
4,vec5,salvación,Juan 3:36,"El que cree en el Hijo, tiene vida eterna; mas...",Este versículo subraya la importancia de la f...,"[Misión de vida, Desarrollo/crecimiento person..."


In [43]:
# Using OpenAI API to do the embeddings
client = OpenAI()

def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

bible_data['texto_vector'] = bible_data.texto.apply(lambda x: get_embedding(x, model='text-embedding-3-small'))
bible_data['interpretacion_vector'] = bible_data.interpretacion.apply(lambda x: get_embedding(x, model='text-embedding-3-small'))
bible_data.to_csv('bible/data/bible_by_theme_int_embedding.csv', index=False)

## Creating Collection in VectorDB

### Chroma DB

https://github.com/openai/openai-cookbook/blob/main/examples/vector_databases/chroma/Using_Chroma_for_embeddings_search.ipynb

https://github.com/openai/openai-cookbook/blob/main/examples/vector_databases/chroma/hyde-with-chroma-and-openai.ipynb

In [ ]:
#### When installing ChromaDB library it installs numpy-1.26.4 (deletes numpy 2.0.1). After installing ChromaDB library restart the kernel
#https://pypi.org/project/numpy/1.26.4/
#https://pypi.org/project/chromadb/0.5.5/

In [1]:
import pandas as pd
import os

os.chdir('/Users/hiroshi/Documents/ds_projects/orare-model')

bible_data = pd.read_csv('bible/data/bible_by_theme_int_embedding.csv')

In [2]:
bible_data.head()

,id,tema,versiculo,texto,interpretacion,area_vida,texto_vector,interpretacion_vector
0,vec1,salvación,Mateo 18:3,"Y dijo: De cierto os digo, que si no os volvie...",Jesús está diciendo que debemos tener la humi...,"['Desarrollo/crecimiento personal', 'Misión de...","[0.023037778213620186, 0.024053120985627174, -...","[0.03883896768093109, 0.0037278474774211645, -..."
1,vec2,salvación,Juan 1:12,"Mas á todos los que le recibieron, dióles pote...",Este versículo significa que a todas las pers...,"['Desarrollo/crecimiento personal', 'Misión de...","[0.06051323190331459, 0.038999710232019424, -0...","[0.0385851114988327, 0.054991379380226135, -0...."
2,vec3,salvación,Juan 3:3,"Respondió Jesús, y díjole: De cierto, de ciert...",Jesús está hablando de la necesidad de un ren...,"['Desarrollo/crecimiento personal', 'Misión de...","[-0.013897780328989029, 0.048658277839422226, ...","[0.017310068011283875, 0.03643307089805603, 0...."
3,vec4,salvación,Juan 3:16,"Porque de tal manera amó Dios al mundo, que ha...",Este versículo expresa el amor inmenso de Dio...,"['Misión de vida', 'Desarrollo/crecimiento per...","[0.03011396899819374, 0.014558355323970318, -0...","[0.030949406325817108, 0.02243080735206604, 0...."
4,vec5,salvación,Juan 3:36,"El que cree en el Hijo, tiene vida eterna; mas...",Este versículo subraya la importancia de la f...,"['Misión de vida', 'Desarrollo/crecimiento per...","[0.000544808164704591, 0.0411251038312912, 0.0...","[0.008498639799654484, 0.0067153871059417725, ..."


In [3]:
import openai
import pandas as pd
import os
from ast import literal_eval

# Chroma's client library for Python
import chromadb

# Ignore unclosed SSL socket warnings - optional in case you get these errors
#import warnings
#warnings.filterwarnings(action="ignore", message="unclosed", category=ResourceWarning)
#warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [4]:
# Read vectors from strings back into a list
#sample_df['interpretacion_vector'] = sample_df.interpretacion_vector.apply(literal_eval)
bible_data.info(show_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 945 entries, 0 to 944
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     945 non-null    object
 1   tema                   945 non-null    object
 2   versiculo              945 non-null    object
 3   texto                  945 non-null    object
 4   interpretacion         945 non-null    object
 5   area_vida              945 non-null    object
 6   texto_vector           945 non-null    object
 7   interpretacion_vector  945 non-null    object
dtypes: object(8)
memory usage: 59.2+ KB


In [5]:
# Instantiate the Chroma client
chroma_client = chromadb.EphemeralClient() # Equivalent to chromadb.Client(), ephemeral.

In [6]:
# Create collections
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

if os.getenv("OPENAI_API_KEY") is not None:
    openai.api_key = os.getenv("OPENAI_API_KEY")
    print ("OPENAI_API_KEY is ready")
else:
    print ("OPENAI_API_KEY environment variable not found")

EMBEDDING_MODEL = "text-embedding-3-small"

embedding_function = OpenAIEmbeddingFunction(api_key=os.environ.get('OPENAI_API_KEY'), model_name=EMBEDDING_MODEL)



OPENAI_API_KEY is ready


In [7]:
# Creating or retrieving the collection in ChromaDB
try:
    # Try to create the collection (this will raise an error if it already exists)
    bible_interpretation_collection = chroma_client.create_collection(name='bible_interpretation', embedding_function=embedding_function)
except Exception as e:
    # If an error occurs (assuming it's because the collection already exists), retrieve it
    if 'Collection bible_interpretation already exists' in str(e):
        bible_interpretation_collection = chroma_client.get_collection(name='bible_interpretation')
    else:
        raise e


In [8]:
# Tranforming vector string into list
import ast
bible_data['interpretacion_vector'] = bible_data['interpretacion_vector'].apply(ast.literal_eval)

# Ahora intenta agregar los embeddings al collection nuevamente.
bible_interpretation_collection.add(
    ids=bible_data.id.tolist(),
    embeddings=bible_data.interpretacion_vector.tolist()
)

In [9]:
# Search the collection
def query_collection(collection, query, max_results, dataframe):
    results = collection.query(query_texts=query, n_results=max_results, include=['distances']) 
    df = pd.DataFrame({
                'id':results['ids'][0], 
                'score':results['distances'][0],
                'tema': dataframe[dataframe.id.isin(results['ids'][0])]['tema'],
                'versiculo': dataframe[dataframe.id.isin(results['ids'][0])]['versiculo'],
                'texto': dataframe[dataframe.id.isin(results['ids'][0])]['texto'],
                'interpretacion': dataframe[dataframe.id.isin(results['ids'][0])]['interpretacion'],
                })
    
    return df

### Testing the retrieval from the vector database

In [10]:
prayer = "Dios, te pido y oro por los pobres del mundo que no tienen que comer. Por favor se misericordioso con ellos. Pido por ellos. Amen"

title_query_result = query_collection(
    collection=bible_interpretation_collection,
    query=prayer,
    max_results=3,
    dataframe=bible_data
)

print(f"Oración: {prayer}")
print('---------------------------')
print(title_query_result.iloc[0,3] + ': ' + title_query_result.iloc[0,4])
print('Interpretación: ' + title_query_result.iloc[0,5])
print('---------------------------')
print(title_query_result.iloc[1,3] + ': ' + title_query_result.iloc[1,4])
print('Interpretación:' + title_query_result.iloc[1,5])
print('---------------------------')
print(title_query_result.iloc[2,3] + ': ' + title_query_result.iloc[2,4])
print('Interpretación:' + title_query_result.iloc[2,5])

Oración: Dios, te pido y oro por los pobres del mundo que no tienen que comer. Por favor se misericordioso con ellos. Pido por ellos. Amen
---------------------------
Hebreos 4:16: Lleguémonos pues confiadamente al trono de la gracia, para alcanzar misericordia, y hallar gracia para el oportuno socorro.
Interpretación:  Acerquémonos con confianza al trono de la gracia de Dios para recibir misericordia y encontrar la ayuda que necesitamos en el momento oportuno. 
---------------------------
Exodo 23:25: Mas á Jehová vuestro Dios serviréis, y él bendecirá tu pan y tus aguas; y yo quitaré toda enfermedad de en medio de ti.
Interpretación: Sirvan al Señor su Dios, y Él bendecirá su alimento y su agua; y yo quitaré toda enfermedad de en medio de ustedes. 
---------------------------
Filipenses 4:6: Por nada estéis afanosos; sino sean notorias vuestras peticiones delante de Dios en toda oración y ruego, con hacimiento de gracias.
Interpretación: No se preocupen por nada; en lugar de eso, pre

In [11]:
prayer = '''Querido Dios, 
Hoy me encuentro angustiado y ansioso por recibir un email que me puede cambiar la vida. Estoy a la espera de una noticia muy importante para mí, pero mi falta de paciencia y de quererlo todo ahora ya, hace que me angustie y estrese. 
Por otro lado, hoy he vuelto ir a la fisio por un dolor que tuve en la pierna y ya estoy mejor, el estres hace que me tense mucho y mis musculos acaben contracturandose demasio.
Te pido que me des paciencia en este momento, que tenga confianza en que todo va a salir bien, y lo que tú quieres para mí se va cumplir tal y como tú lo planeas.
Amén.'''

title_query_result = query_collection(
    collection=bible_interpretation_collection,
    query=prayer,
    max_results=3,
    dataframe=bible_data
)

print(f"Oración: {prayer}")
print('---------------------------')
print(title_query_result.iloc[0,3] + ': ' + title_query_result.iloc[0,4])
print('Interpretación: ' + title_query_result.iloc[0,5])
print('---------------------------')
print(title_query_result.iloc[1,3] + ': ' + title_query_result.iloc[1,4])
print('Interpretación:' + title_query_result.iloc[1,5])
print('---------------------------')
print(title_query_result.iloc[2,3] + ': ' + title_query_result.iloc[2,4])
print('Interpretación:' + title_query_result.iloc[2,5])

Oración: Querido Dios, 
Hoy me encuentro angustiado y ansioso por recibir un email que me puede cambiar la vida. Estoy a la espera de una noticia muy importante para mí, pero mi falta de paciencia y de quererlo todo ahora ya, hace que me angustie y estrese. 
Por otro lado, hoy he vuelto ir a la fisio por un dolor que tuve en la pierna y ya estoy mejor, el estres hace que me tense mucho y mis musculos acaben contracturandose demasio.
Te pido que me des paciencia en este momento, que tenga confianza en que todo va a salir bien, y lo que tú quieres para mí se va cumplir tal y como tú lo planeas.
Amén.
---------------------------
Salmos 94:19: En la multitud de mis pensamientos dentro de mí, Tus consolaciones alegraban mi alma.
Interpretación:  En tiempos de preocupación y ansiedad, el consuelo de Dios trae alegría y paz al alma. 
---------------------------
Salmos 119:28: Deshácese mi alma de ansiedad: Corrobórame según tu palabra.
Interpretación: Mi alma se siente abrumada por la ansieda

In [12]:
prayer = "Hola, qué tal?"

title_query_result = query_collection(
    collection=bible_interpretation_collection,
    query=prayer,
    max_results=3,
    dataframe=bible_data
)

print(f"Oración: {prayer}")
print('---------------------------')
print(title_query_result.iloc[0,3] + ': ' + title_query_result.iloc[0,4])
print('Interpretación: ' + title_query_result.iloc[0,5])
print('---------------------------')
print(title_query_result.iloc[1,3] + ': ' + title_query_result.iloc[1,4])
print('Interpretación:' + title_query_result.iloc[1,5])
print('---------------------------')
print(title_query_result.iloc[2,3] + ': ' + title_query_result.iloc[2,4])
print('Interpretación:' + title_query_result.iloc[2,5])

Oración: Hola, qué tal?
---------------------------
Salmos 27:14: Aguarda á Jehová; Esfuérzate, y aliéntese tu corazón: Sí, espera á Jehová.
Interpretación:  Aguarda al Señor; esfuérzate y aliéntese tu corazón: sí, espera al Señor. 
---------------------------
Salmos 34:4: Busqué á Jehová, y él me oyó, Y libróme de todos mis temores.
Interpretación: Busqué al Señor, y él me escuchó, y me libró de todos mis temores. 
---------------------------
Isaías 50:4: El Señor Jehová me dió lengua de sabios, para saber hablar en sazón palabra al cansado; despertará de mañana, despertaráme de mañana oído, para que oiga como los sabios.
Interpretación: El Señor Dios me ha dado una lengua instruida, para saber decir al cansado una palabra de aliento; cada mañana me despierta, me despierta el oído para que escuche como los discípulos. 


In [13]:
prayer = '''Dios en esta oportunidad quiero pedirte que me des luz y fuerzas para poder continuar con la búsqueda de trabajo que estoy haciendo. Se que es un poco dificil conseguir trabajo como extranjero, pero no tengo dudas de que Tú me ayudarás y pondrás la oportunidad ante mis ojos. En el nombre del Padre, del Hijo, del Espíritu Santo, Amén.'''

title_query_result = query_collection(
    collection=bible_interpretation_collection,
    query=prayer,
    max_results=3,
    dataframe=bible_data
)

print(f"Oración: {prayer}")
print('---------------------------')
print(title_query_result.iloc[0,3] + ': ' + title_query_result.iloc[0,4])
print('Interpretación: ' + title_query_result.iloc[0,5])
print('---------------------------')
print(title_query_result.iloc[1,3] + ': ' + title_query_result.iloc[1,4])
print('Interpretación:' + title_query_result.iloc[1,5])
print('---------------------------')
print(title_query_result.iloc[2,3] + ': ' + title_query_result.iloc[2,4])
print('Interpretación:' + title_query_result.iloc[2,5])

Oración: Dios en esta oportunidad quiero pedirte que me des luz y fuerzas para poder continuar con la búsqueda de trabajo que estoy haciendo. Se que es un poco dificil conseguir trabajo como extranjero, pero no tengo dudas de que Tú me ayudarás y pondrás la oportunidad ante mis ojos. En el nombre del Padre, del Hijo, del Espíritu Santo, Amén.
---------------------------
Efesios 3:16: Que os dé, conforme á las riquezas de su gloria, el ser corroborados con potencia en el hombre interior por su Espíritu.
Interpretación:  Que Dios, conforme a las riquezas de su gloria, les fortalezca con poder en su ser interior por medio de su Espíritu. 
---------------------------
Salmos 18:29, 30B, 32: Porque contigo desharé ejércitos; Y con mi Dios asaltaré muros. Dios, perfecto su camino: Es acendrada la palabra de Jehová: Escudo es á todos los que en él esperan. Dios es el que me ciñe de fuerza, E hizo perfecto mi camino;
Interpretación: Con la ayuda de Dios, puedo superar grandes obstáculos y desaf

In [14]:
prayer = '''Le pido a Dios me de claridad en cuanto a mi objetivo a largo plazo profesional/laboral'''

title_query_result = query_collection(
    collection=bible_interpretation_collection,
    query=prayer,
    max_results=3,
    dataframe=bible_data
)

print(f"Oración: {prayer}")
print('---------------------------')
print(title_query_result.iloc[0,3] + ': ' + title_query_result.iloc[0,4])
print('Interpretación: ' + title_query_result.iloc[0,5])
print('---------------------------')
print(title_query_result.iloc[1,3] + ': ' + title_query_result.iloc[1,4])
print('Interpretación:' + title_query_result.iloc[1,5])
print('---------------------------')
print(title_query_result.iloc[2,3] + ': ' + title_query_result.iloc[2,4])
print('Interpretación:' + title_query_result.iloc[2,5])

Oración: Le pido a Dios me de claridad en cuanto a mi objetivo a largo plazo profesional/laboral
---------------------------
Salmos 25:4, 5: Muéstrame, oh Jehová, tus caminos; Enséñame tus sendas.  Encamíname en tu verdad, y enséñame; Porque tú eres el Dios de mi salud: En ti he esperado todo el día.
Interpretación:  El salmista está pidiendo a Dios que le muestre el camino correcto y le enseñe la verdad. Está buscando guía y dirección divina en su vida, confiando en Dios para su bienestar y salvación. 
---------------------------
Salmos 119:18: Abre mis ojos, y miraré Las maravillas de tu ley.
Interpretación: El salmista está pidiendo a Dios que le dé entendimiento y claridad para poder apreciar y comprender las maravillas y enseñanzas contenidas en la ley divina. 
---------------------------
Salmos 143:10, 11: Enséñame á hacer tu voluntad, porque tú eres mi Dios: Tu buen espíritu me guíe á tierra de rectitud. Por tu nombre, oh Jehová me vivificarás: Por tu justicia, sacarás mi alma d

In [36]:
oracion = '''No se que podrá pasar mañana, pero confío en ti. Nunca he estado en tanta sintonía con mi presente, con esta adrenalina de ver la primera cima, de empezar a estirar los brazos para cruzar la meta y de esta fe ciega de que las cosas se van a ir poniendo en su lugar. Hoy la vida me vuelve a enseñar, que nada está en mi control, que todo depende de ti. Y por eso acudo una vez más a tu consejo, guíame señor, dame esa luz para poder convencer si son esos tus planes, muéstrame el camino, dame la esperanza para luchar esta batalla una vez más, para inflar el pecho por creer que todo es posible, pero que es posible junto a ti. Pero te pido también, con las manos temblando del temor, que si esta puerta no es para mi, la cierres por completo, aleja de mi lo que a la larga no sea para tu provecho, si este sendero no he de recorrer, cámbiame el camino, quiebra la oportunidad, porque prefiero perder incluso mis sueños, pero jamás perderte a ti, porque una vida de triunfos es vacío si no estás a mi lado, porque aún en la derrota, hay paz si es estás junto a mi, porque hoy más que nunca, te digo de rodillas, que se haga en mi según tu palabra, que se haga en mi según tu voluntad y te ruego me des la sabiduría para entender, cualquiera que sea tu designio. Amén'''

title_query_result = query_collection(
    collection=bible_interpretation_collection,
    query=oracion,
    max_results=3,
    dataframe=bible_data
)

print(f"Oración: {oracion}")
print('---------------------------')
print(title_query_result.iloc[0,3] + ': ' + title_query_result.iloc[0,4])
print('interpretación: ' + title_query_result.iloc[0,5])
print('---------------------------')
print(title_query_result.iloc[1,3] + ': ' + title_query_result.iloc[1,4])
print('interpretación:' + title_query_result.iloc[1,5])
print('---------------------------')
print(title_query_result.iloc[2,3] + ': ' + title_query_result.iloc[2,4])
print('interpretación:' + title_query_result.iloc[2,5])

Oración: No se que podrá pasar mañana, pero confío en ti. Nunca he estado en tanta sintonía con mi presente, con esta adrenalina de ver la primera cima, de empezar a estirar los brazos para cruzar la meta y de esta fe ciega de que las cosas se van a ir poniendo en su lugar. Hoy la vida me vuelve a enseñar, que nada está en mi control, que todo depende de ti. Y por eso acudo una vez más a tu consejo, guíame señor, dame esa luz para poder convencer si son esos tus planes, muéstrame el camino, dame la esperanza para luchar esta batalla una vez más, para inflar el pecho por creer que todo es posible, pero que es posible junto a ti. Pero te pido también, con las manos temblando del temor, que si esta puerta no es para mi, la cierres por completo, aleja de mi lo que a la larga no sea para tu provecho, si este sendero no he de recorrer, cámbiame el camino, quiebra la oportunidad, porque prefiero perder incluso mis sueños, pero jamás perderte a ti, porque una vida de triunfos es vacío si no es

In [15]:
prayer = '''Dios. Quiero que en este proceso de búsqueda seas mi fuerza y me acompañes en todo momento. 
Yo se que tienes para mi un plan. Yo se que tu siempre das en la medida correcta y que poco a poco me has mostrado el camino para mi crecimiento.
Hace un tiempo me faltaba confianza en mi mismo. Se que con la experiencia laboral anterior me diste la oportunidad de abrir los ojos y saber quien soy.
Te doy las gracias porque se que cada paso que me muestras es un paso hacia desarrollar mi máximo potencial.
'''

title_query_result = query_collection(
    collection=bible_interpretation_collection,
    query=prayer,
    max_results=3,
    dataframe=bible_data
)

print(f"Oración: {prayer}")
print('---------------------------')
print(title_query_result.iloc[0,3] + ': ' + title_query_result.iloc[0,4])
print('Interpretación: ' + title_query_result.iloc[0,5])
print('---------------------------')
print(title_query_result.iloc[1,3] + ': ' + title_query_result.iloc[1,4])
print('Interpretación:' + title_query_result.iloc[1,5])
print('---------------------------')
print(title_query_result.iloc[2,3] + ': ' + title_query_result.iloc[2,4])
print('Interpretación:' + title_query_result.iloc[2,5])

Oración: Dios. Quiero que en este proceso de búsqueda seas mi fuerza y me acompañes en todo momento. 
Yo se que tienes para mi un plan. Yo se que tu siempre das en la medida correcta y que poco a poco me has mostrado el camino para mi crecimiento.
Hace un tiempo me faltaba confianza en mi mismo. Se que con la experiencia laboral anterior me diste la oportunidad de abrir los ojos y saber quien soy.
Te doy las gracias porque se que cada paso que me muestras es un paso hacia desarrollar mi máximo potencial.

---------------------------
Salmos 18:29, 30B, 32: Porque contigo desharé ejércitos; Y con mi Dios asaltaré muros. Dios, perfecto su camino: Es acendrada la palabra de Jehová: Escudo es á todos los que en él esperan. Dios es el que me ciñe de fuerza, E hizo perfecto mi camino;
Interpretación:  Con la ayuda de Dios, puedo superar grandes obstáculos y desafíos. La palabra de Dios es pura y perfecta, y Él protege a quienes confían en Él. Dios me da la fuerza y guía para seguir el camino 

In [16]:
prayer = '''Querido Dios,
Hoy ha sido un gran día. He vuelto de un viaje de Madrid con mi novia, y ha ido genial me lo he pasado muy bien, 
aunque he estado cansado durante el día. Y he tenido la noticia, de que ya empezaré a trabajar en una nueva empresa en septiembre,
con la cual estoy muy motivado y con muchas ganas de empezar. Esto significa, que debo de poner la renuncia en la actual empresa,
dame la serenidad y calma para hacerlo de la mejor manera dando gracias por la oportunidad que me han dado, 
pero se me presentó una oportunidad irrechazable. Espero que no se lo tomen mal, intentaré decirles con las mejores palabras y respeto. 
También, hoy ha sido el cumple de mi padre, hemos pasado una bonita cena en familia, con los míos.
Muchas gracias por otro día más de vida'''

title_query_result = query_collection(
    collection=bible_interpretation_collection,
    query=prayer,
    max_results=3,
    dataframe=bible_data
)

print(f"Oración: {prayer}")
print('---------------------------')
print(title_query_result.iloc[0,3] + ': ' + title_query_result.iloc[0,4])
print('Interpretación: ' + title_query_result.iloc[0,5])
print('---------------------------')
print(title_query_result.iloc[1,3] + ': ' + title_query_result.iloc[1,4])
print('Interpretación:' + title_query_result.iloc[1,5])
print('---------------------------')
print(title_query_result.iloc[2,3] + ': ' + title_query_result.iloc[2,4])
print('Interpretación:' + title_query_result.iloc[2,5])

Oración: Querido Dios,
Hoy ha sido un gran día. He vuelto de un viaje de Madrid con mi novia, y ha ido genial me lo he pasado muy bien, 
aunque he estado cansado durante el día. Y he tenido la noticia, de que ya empezaré a trabajar en una nueva empresa en septiembre,
con la cual estoy muy motivado y con muchas ganas de empezar. Esto significa, que debo de poner la renuncia en la actual empresa,
dame la serenidad y calma para hacerlo de la mejor manera dando gracias por la oportunidad que me han dado, 
pero se me presentó una oportunidad irrechazable. Espero que no se lo tomen mal, intentaré decirles con las mejores palabras y respeto. 
También, hoy ha sido el cumple de mi padre, hemos pasado una bonita cena en familia, con los míos.
Muchas gracias por otro día más de vida
---------------------------
Salmos 71:17, 18: Oh Dios, enseñásteme desde mi mocedad; Y hasta ahora he manifestado tus maravillas. Y aun hasta la vejez y las canas; oh Dios, no me desampares, Hasta que denuncie tu braz

In [32]:
oracion = '''Señor bendito esta mañana t agradezco la salud y la vida
Que tu gracia y misericordia sean hoy conmigo 
Padre t ruego fortaleza  en todo momento 
Que tu luz siempre  m acompañe
Perdóname en mis momentos d cólera  
Ayúdame a tener paciencia  y amor para mi y mis semejantes '''

title_query_result = query_collection(
    collection=bible_interpretation_collection,
    query=oracion,
    max_results=3,
    dataframe=bible_data
)

print(f"Oración: {oracion}")
print('---------------------------')
print(title_query_result.iloc[0,3] + ': ' + title_query_result.iloc[0,4])
print('Interpretación: ' + title_query_result.iloc[0,5])
print('---------------------------')
print(title_query_result.iloc[1,3] + ': ' + title_query_result.iloc[1,4])
print('Interpretación:' + title_query_result.iloc[1,5])
print('---------------------------')
print(title_query_result.iloc[2,3] + ': ' + title_query_result.iloc[2,4])
print('Interpretación:' + title_query_result.iloc[2,5])

Oración: Señor bendito esta mañana t agradezco la salud y la vida
Que tu gracia y misericordia sean hoy conmigo 
Padre t ruego fortaleza  en todo momento 
Que tu luz siempre  m acompañe
Perdóname en mis momentos d cólera  
Ayúdame a tener paciencia  y amor para mi y mis semejantes 
---------------------------
Salmos 116:1, 2: AMO á Jehová, pues ha oído Mi voz y mis súplicas. Porque ha inclinado á mí su oído, Invocaré le por tanto en todos mis días.
Interpretación:  Amo al Señor porque ha escuchado mi voz y mis súplicas. Porque ha inclinado su oído hacia mí, lo invocaré todos los días de mi vida. 
---------------------------
Salmos 68:19: Bendito el Señor; cada día nos colma de beneficios El Dios de nuestra salud. (Selah.)
Interpretación: Bendito el Señor; cada día nos colma de beneficios. El Dios de nuestra salud. (Selah.) 
---------------------------
2 Colosenses 3:5: Y el Señor enderece vuestros corazones en el amor de Dios, y en la paciencia de Cristo.
Interpretación: Que el Señor g

In [17]:
oracion = '''Ilumíname señor, dame paz en estos momentos de incertidumbre, sujétame de tu mano, permíteme caminar junto a ti.
¿Por qué permites este dolor dentro de nosotros? ¿Por qué este sufrimiento tan profundo? No lograré entender tus motivos, 
pero entiendo espiritualmente lo que estás haciendo conmigo.
Purificas mi alma, me permitiste soltar el control, afianzarme a ti, volver a mirarte cara a cara, 
entender que todo depende de ti, reconocer la vanidad de mis actos, sentir miedo a cada instante y que mi única fuerza seas tú.
No entiendo tus caminos pero decido caminar dentro de ellos, seguir tu luz, guiar a mi familia hacia tus designios, 
respirar en esa absoluta incertidumbre y orar con todas mis fuerzas para que se haga según tu voluntad.
Cuida a mi familia señor, bendice a mi esposa, cúbrela con tu manto sagrado, fortalece a mi hijo, 
aliéntalo con tu espíritu, dale el soporte que hoy nosotros no podemos darle, 
que su angel guardián le cuente que estamos todos pendientes de él, 
que no contamos los minutos para tenerlo en nuestros brazos, pero que sea paciente,
que tú tienes varios corazones que limpiar en estos momentos, que su venida está generando santidad en nuestro alrededor.'''

title_query_result = query_collection(
    collection=bible_interpretation_collection,
    query=prayer,
    max_results=3,
    dataframe=bible_data
)

print(f"Oración: {prayer}")
print('---------------------------')
print(title_query_result.iloc[0,3] + ': ' + title_query_result.iloc[0,4])
print('Interpretación: ' + title_query_result.iloc[0,5])
print('---------------------------')
print(title_query_result.iloc[1,3] + ': ' + title_query_result.iloc[1,4])
print('Interpretación:' + title_query_result.iloc[1,5])
print('---------------------------')
print(title_query_result.iloc[2,3] + ': ' + title_query_result.iloc[2,4])
print('Interpretación:' + title_query_result.iloc[2,5])

Oración: Querido Dios,
Hoy ha sido un gran día. He vuelto de un viaje de Madrid con mi novia, y ha ido genial me lo he pasado muy bien, 
aunque he estado cansado durante el día. Y he tenido la noticia, de que ya empezaré a trabajar en una nueva empresa en septiembre,
con la cual estoy muy motivado y con muchas ganas de empezar. Esto significa, que debo de poner la renuncia en la actual empresa,
dame la serenidad y calma para hacerlo de la mejor manera dando gracias por la oportunidad que me han dado, 
pero se me presentó una oportunidad irrechazable. Espero que no se lo tomen mal, intentaré decirles con las mejores palabras y respeto. 
También, hoy ha sido el cumple de mi padre, hemos pasado una bonita cena en familia, con los míos.
Muchas gracias por otro día más de vida
---------------------------
Salmos 71:17, 18: Oh Dios, enseñásteme desde mi mocedad; Y hasta ahora he manifestado tus maravillas. Y aun hasta la vejez y las canas; oh Dios, no me desampares, Hasta que denuncie tu braz

In [34]:
oracion = '''Ilumíname señor, dame paz en estos momentos de incertidumbre, sujétame de tu mano, permíteme caminar junto a ti. ¿Por qué permites este dolor dentro de nosotros? ¿Por qué este sufrimiento tan profundo? No lograré entender tus motivos, pero entiendo espiritualmente lo que estás haciendo conmigo. Purificas mi alma, me permitiste soltar el control, afianzarme a ti, volver a mirarte cara a cara, entender que todo depende de ti, reconocer la vanidad de mis actos, sentir miedo a cada instante y que mi única fuerza seas tú. No entiendo tus caminos pero decido caminar dentro de ellos, seguir tu luz, guiar a mi familia hacia tus designios, respirar en esa absoluta incertidumbre y orar con todas mis fuerzas para que se haga según tu voluntad.Cuida a mi familia señor, bendice a mi esposa, cúbrela con tu manto sagrado, fortalece a mi hijo, aliéntalo con tu espíritu, dale el soporte que hoy nosotros no podemos darle, que su angel guardián le cuente que estamos todos pendientes de él, que no contamos los minutos para tenerlo en nuestros brazos, pero que sea paciente,que tú tienes varios corazones que limpiar en estos momentos, que su venida está generando santidad en nuestro alrededor.'''

title_query_result = query_collection(
    collection=bible_interpretation_collection,
    query=oracion,
    max_results=5,
    dataframe=bible_data
)

print(f"Oración: {oracion}")
print('---------------------------')
print(title_query_result.iloc[0,3] + ': ' + title_query_result.iloc[0,4])
print('Interpretación: ' + title_query_result.iloc[0,5])
print('---------------------------')
print(title_query_result.iloc[1,3] + ': ' + title_query_result.iloc[1,4])
print('Interpretación:' + title_query_result.iloc[1,5])
print('---------------------------')
print(title_query_result.iloc[2,3] + ': ' + title_query_result.iloc[2,4])
print('Interpretación:' + title_query_result.iloc[2,5])
print('---------------------------')
print(title_query_result.iloc[3,3] + ': ' + title_query_result.iloc[3,4])
print('Interpretación:' + title_query_result.iloc[3,5])
print('---------------------------')
print(title_query_result.iloc[4,3] + ': ' + title_query_result.iloc[4,4])
print('Interpretación:' + title_query_result.iloc[4,5])

Oración: Ilumíname señor, dame paz en estos momentos de incertidumbre, sujétame de tu mano, permíteme caminar junto a ti. ¿Por qué permites este dolor dentro de nosotros? ¿Por qué este sufrimiento tan profundo? No lograré entender tus motivos, pero entiendo espiritualmente lo que estás haciendo conmigo. Purificas mi alma, me permitiste soltar el control, afianzarme a ti, volver a mirarte cara a cara, entender que todo depende de ti, reconocer la vanidad de mis actos, sentir miedo a cada instante y que mi única fuerza seas tú. No entiendo tus caminos pero decido caminar dentro de ellos, seguir tu luz, guiar a mi familia hacia tus designios, respirar en esa absoluta incertidumbre y orar con todas mis fuerzas para que se haga según tu voluntad.Cuida a mi familia señor, bendice a mi esposa, cúbrela con tu manto sagrado, fortalece a mi hijo, aliéntalo con tu espíritu, dale el soporte que hoy nosotros no podemos darle, que su angel guardián le cuente que estamos todos pendientes de él, que n

In [18]:
prayer = ''' Dios mío gracias por un nuevo día, donde no me falta salud ni a mí ni a los míos. Te pido me des las fuerzas para seguir de pie y no rendirme en este nuevo camino que he tomado fuera de mi país, migrar no es fácil, lo sé, y extraño tanto a mi tierra y los míos que a veces se van las fuerzas de seguir luchando. Estoy aquí a pesar de recibir miles des “NO” y de que me hayan rechazado mi visado para estar aquí en España… será que sigo luchando o tú ya tienes marcado un destino para mí en otro lugar ?'''

title_query_result = query_collection(
    collection=bible_interpretation_collection,
    query=prayer,
    max_results=3,
    dataframe=bible_data
)

print(f"Oración: {prayer}")
print('---------------------------')
print(title_query_result.iloc[0,3] + ': ' + title_query_result.iloc[0,4])
print('Interpretación: ' + title_query_result.iloc[0,5])
print('---------------------------')
print(title_query_result.iloc[1,3] + ': ' + title_query_result.iloc[1,4])
print('Interpretación:' + title_query_result.iloc[1,5])
print('---------------------------')
print(title_query_result.iloc[2,3] + ': ' + title_query_result.iloc[2,4])
print('Interpretación:' + title_query_result.iloc[2,5])

Oración:  Dios mío gracias por un nuevo día, donde no me falta salud ni a mí ni a los míos. Te pido me des las fuerzas para seguir de pie y no rendirme en este nuevo camino que he tomado fuera de mi país, migrar no es fácil, lo sé, y extraño tanto a mi tierra y los míos que a veces se van las fuerzas de seguir luchando. Estoy aquí a pesar de recibir miles des “NO” y de que me hayan rechazado mi visado para estar aquí en España… será que sigo luchando o tú ya tienes marcado un destino para mí en otro lugar ?
---------------------------
Job 13:15: He aquí, aunque me matare, en él esperaré;
Interpretación:  Aunque Dios me quite la vida, seguiré confiando en Él. 
---------------------------
Isaías 40:29: El da esfuerzo al cansado, y multiplica las fuerzas al que no tiene ningunas.
Interpretación: Dios da fuerza y ánimo a aquellos que están cansados y sin energía. 
---------------------------
Salmos 18:29, 30B, 32: Porque contigo desharé ejércitos; Y con mi Dios asaltaré muros. Dios, perfec

# Pinecone

https://github.com/openai/openai-cookbook/blob/main/examples/vector_databases/pinecone/Using_Pinecone_for_embeddings_search.ipynb

In [ ]:
!pip install -qU openai==1.2.2 pinecone-client

### Create Index

In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="API_KEY")

# Create Index
index_name = "text-embedding-3-large"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=3074,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

index = pc.Index(index_name)

### Embed & Upsert

In [ ]:
# Embed data
data = [
    {"id": "vec1", "text": "Apple is a popular fruit known for its sweetness and crisp texture."},
    {"id": "vec2", "text": "The tech company Apple is known for its innovative products like the iPhone."},
    {"id": "vec3", "text": "Many people enjoy eating apples as a healthy snack."},
    {"id": "vec4", "text": "Apple Inc. has revolutionized the tech industry with its sleek designs and user-friendly interfaces."},
    {"id": "vec5", "text": "An apple a day keeps the doctor away, as the saying goes."},
]

import openai 
openai.api_key = "OPENAI_API_KEY" 


def embed(docs: list[str]) -> list[list[float]]:
    res = openai.embeddings.create(
        input=docs,
        model="text-embedding-3-large"
    )
    doc_embeds = [r.embedding for r in res.data] 
    return doc_embeds 


doc_embeds = embed([d["text"] for d in data])

vectors = []
for d, e in zip(data, doc_embeds):
    vectors.append({
        "id": d['id'],
        "values": e,
        "metadata": {'text': d['text']}
    })

index.upsert(
    vectors=vectors,
    namespace="ns1"
)


### Query
query = "Tell me about the tech company known as Apple"

x = embed([query])

results = index.query(
    namespace="ns1",
    vector=x[0],
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)


In [ ]:
## Temas a corregir
# Hay versiculos repetidos. Quitar versiculos duplicados. 
# Revisar las interpretaciones. Hay interpretaciones que son un copia y pega. 